# Boolean Retrieval

In [125]:
%pip install -r requirements.txt
%load_ext autoreload
%autoreload 2

Note: you may need to restart the kernel to use updated packages.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [126]:
from helpers import set_of_words, BooleanRetriever, print_table
from datasets import animals as collection
import ipywidgets as widgets
import random
from itertools import islice

In [127]:
# load data set and index collection for boolean retrieval
retriever = BooleanRetriever(collection.load())

# show collection
n, m = min(10, retriever.n_docs), min(20, retriever.n_terms)

print_table([collection.format(doc) for doc in retriever.documents.values()], collection.headers(), max_rows = n)
print_table(sorted([[term, df, retriever.index[term]] for term, df in retriever.vocabulary.items()], key=lambda x: -x[1]), ['term', 'df', 'posting'], max_rows=20)
print(f'{retriever.n_docs} documents in collection')
print(f'{retriever.n_terms} distinct terms in collection')
print('{count} postings'.format(count=sum([len(postings) for postings in retriever.index.values()])))

|   docId | text                                                             |
|--------:|:-----------------------------------------------------------------|
|       1 | dog dog dog bee                                                  |
|       2 | ostrich ostrich ostrich donkey donkey donkey donkey donkey wale  |
|       3 | dog dog dog dog dog cat cat cat cat cat bear bear bear bear bear |
|       4 | rabit rabit bear bear tiger                                      |
|       5 | bear ant ant ant                                                 |
|       6 | horse horse horse horse horse bear                               |
|       7 | ostrich ostrich ostrich ant ant ant ant fly fly fly fly fly      |
|       8 | horse horse horse ostrich ostrich                                |
|       9 | rabit rabit rabit rabit rabit donkey donkey donkey               |
|      10 | dog lion lion bird bird bird bird bird                           |

| term    |   df | posting                                                                                                                |
|:--------|-----:|:-----------------------------------------------------------------------------------------------------------------------|
| dog     |   30 | {1, 3, 10, 13, 14, 16, 20, 21, 24, 33, 34, 41, 42, 43, 47, 56, 61, 62, 64, 65, 66, 70, 75, 83, 84, 86, 90, 91, 92, 93} |
| cat     |   25 | {3, 12, 13, 15, 19, 22, 30, 31, 34, 36, 37, 50, 52, 53, 56, 60, 66, 67, 85, 91, 92, 96, 97, 98, 100}                   |
| hors    |   22 | {6, 8, 13, 26, 27, 31, 32, 35, 41, 43, 61, 65, 67, 71, 76, 83, 84, 85, 88, 92, 93, 100}                                |
| rabit   |   20 | {4, 9, 14, 19, 20, 22, 27, 34, 41, 43, 47, 53, 59, 64, 65, 70, 76, 77, 81, 97}                                         |
| ostrich |   18 | {2, 34, 67, 7, 8, 39, 40, 42, 13, 78, 15, 81, 61, 83, 53, 22, 59, 29}                                                  |
| bear    |   17 | {33, 65, 3, 4, 5, 6, 36, 67, 68, 70, 75, 12, 77, 19, 83, 59, 95}                                                       |
| tiger   |   15 | {33, 4, 41, 78, 79, 48, 83, 20, 55, 54, 23, 58, 92, 61, 30}                                                            |
| bird    |   14 | {34, 38, 39, 70, 72, 10, 42, 74, 76, 77, 17, 19, 55, 31}                                                               |
| lion    |   14 | {65, 35, 36, 38, 10, 43, 44, 15, 48, 88, 25, 58, 63, 57}                                                               |
| donkey  |   13 | {2, 40, 9, 91, 13, 77, 15, 24, 25, 26, 27, 61, 31}                                                                     |
| bee     |   12 | {1, 65, 99, 75, 16, 81, 83, 85, 55, 92, 94, 31}                                                                        |
| ant     |   12 | {99, 36, 5, 100, 7, 76, 80, 51, 87, 56, 90, 93}                                                                        |
| wale    |   11 | {33, 2, 34, 66, 38, 75, 92, 20, 24, 60, 30}                                                                            |
| fli     |   11 | {66, 38, 7, 43, 12, 14, 48, 85, 88, 58, 92}                                                                            |
| snake   |   10 | {99, 69, 43, 44, 77, 55, 25, 93, 62, 95}                                                                               |
| fish    |    9 | {73, 11, 45, 46, 49, 18, 82, 89, 28}                                                                                   |

100 documents in collection
16 distinct terms in collection
253 postings


## Set operations to answer queries

The next section demonstrates how to perform Boolean queries using the inverted index with set operations. 

| Boolean Operator | Set Operator for Postings |
| :--- | :--- |
| cat AND dog | `index['cat'] & index['dog']` |
| cat OR dog | `index['cat'] \| index['dog']` |
| cat AND NOT dog | `index['cat'] - index['dog']` |

Using these rules, we can evaluate a wide range of Boolean queries. However, there are some limitations:
- It is inefficient to evaluate OR-queries when one sub-expression is of the form NOT(expr)
- In AND-clauses, NOT(expr)-parts need to be re-ordered to the end to apply the `-` set operator. If all AND operators have NOT(expr)-parts, evaluation becomes inefficient
- In both cases, the NOT(expr) part can be replaced with (ALL - expr), where ALL represents all documents

Indeed, while these limitations may be viewed as constraints in our implementation, they have minimal impact on practical querying scenarios.

In [128]:
# base sets
all = retriever.all
cat = retriever.index['cat']
dog = retriever.index['dog']
horse = retriever.index['horse']
bird = retriever.index['bird']

# cat AND dog
print(f'cat AND dog: {cat & dog}')

# cat OR horse
print(f'cat or horse: {cat | horse}')

# cat AND NOT(dog)
print(f'cat or horse: {cat - dog}')

cat AND dog: {66, 34, 3, 13, 56, 91, 92}
cat or horse: {66, 3, 67, 12, 13, 15, 19, 85, 22, 91, 92, 30, 31, 96, 97, 34, 98, 36, 37, 100, 50, 52, 53, 56, 60}
cat or horse: {96, 97, 98, 67, 36, 37, 100, 12, 15, 50, 19, 52, 53, 22, 85, 60, 30, 31}


In [129]:
results = {
    'cat': cat,
    'dog': dog,
    'horse': horse,
    'bird': bird,
    'NOT(dog)': all - dog,
    'NOT(cat)': all - cat,
    'NOT(cat) AND NOT(dog)': (all - dog) & (all - cat),
    'cat OR NOT(dog)': cat | (all - dog), 
    'cat AND dog': cat & dog,
    'cat OR dog': cat | dog,
    'horse OR bird': horse | bird,
    'cat AND NOT(dog)': cat - dog,
    'horse AND cat AND NOT(bird)': horse & cat - bird,
    'horse AND cat': horse & cat,
    '(cat AND dog) OR (horse AND cat AND NOT(bird))': (cat & dog) | (horse & cat - bird),
    'horse OR bird': horse | bird,
    '(cat OR dog) AND (horse OR bird)': (cat | dog) & (horse | bird),
    '(cat OR dog) AND NOT(horse OR bird)': (cat | dog) - (horse | bird)
}

queries = {
    'cat AND dog': ['cat', 'dog', 'cat AND dog'],
    'cat OR dog': ['cat', 'dog', 'cat OR dog'],
    'horse OR bird': ['horse', 'bird', 'horse OR bird'],
    'cat AND NOT(dog)': ['cat', 'dog', 'NOT(dog)', 'cat AND NOT(dog)'],
    'cat OR NOT(dog)': ['cat', 'dog', 'NOT(dog)', 'cat OR NOT(dog)'],
    'NOT(cat) AND NOT(dog)': ['cat', 'dog', 'NOT(cat)', 'NOT(dog)', 'NOT(cat) AND NOT(dog)'],
    'horse AND cat AND NOT(bird)': ['horse', 'cat', 'bird', 'horse AND cat', 'horse AND cat AND NOT(bird)'],
    '(cat AND dog) OR (horse AND cat AND NOT(bird))': ['cat', 'dog', 'horse', 'bird', 'cat AND dog', 'horse AND cat', 'horse AND cat AND NOT(bird)', '(cat AND dog) OR (horse AND cat AND NOT(bird))'],
    '(cat OR dog) AND (horse OR bird)': ['cat', 'dog', 'cat OR dog', 'horse OR bird', '(cat OR dog) AND (horse OR bird)'],
    '(cat OR dog) AND NOT(horse OR bird)': ['cat', 'dog', 'cat OR dog', 'horse OR bird', '(cat OR dog) AND NOT(horse OR bird)']    
}

def search_boolean_set(query: str):
    print_table([[q, sorted(results[q])] for q in queries[query]], ['query', 'result'], format="text")

# interactive selection of scenario
widgets.interact(search_boolean_set, 
    query=widgets.Dropdown(options=list(queries.keys())), 
);

interactive(children=(Dropdown(description='query', options=('cat AND dog', 'cat OR dog', 'horse OR bird', 'ca…

## Stream operations to answer queries

The set-based evaluation from above does not scale well with the number of documents. In cases with millions of billions of postings for a term, we want to fetch data from an external storage device (which is also a good idea for persistence). But instead of reading all postings into main memory, we read them as streams sorted by the document IDs. Take the postings of cat and dog as an example:

| term | postings|
| :-- | :-- |
| cat | `[1, 4, 8, 10]` |
| dog | `[3, 4, 10, 12]` |

### A representation for Boolean expressions
Next, we build a representation for Boolean expressions that allows us to retrieve arbitrary queries. Each object in the representation relates to a sub-expression in the query that is evaluated in a stream-based manner. By composition, we can aggregate atomic queries over terms to arbitrarily complex Boolean queries. We introduce a class ``Term`` to denote atomic queries, 2 classes ``And`` and ``Or`` that represent the boolean operators with an arbitrary number of operands (which can be of type ``Term``, ``And``, and ``Or``). Finally, we also define a class ``Not`` that acts on a single sub-expression (again of any sub-type). However, we do not allow nesting of ``Not``, and only ``And`` expression can have a sub-expression with ``Not`` (and must have at leas one subexpression without ``Not``). Our implentation of ``Not`` is to mark sub-expression, but the class has no way to evaluate itself.

In [130]:
class BooleanExpression:
    pass

# ------------------------------

class Term(BooleanExpression):
    """
        Boolean expression class for atomic term queries. For simplicity, we
        have all postings in memory but in real implementations, we would
        fetch the data from a file or database
    """
    def __init__(self, term: str, postings: list[int]):
        self.term = term
        self.postings = sorted(postings)

    def __iter__(self):
        return self.retrieve()
    
    # we used this flag to monitor which postings we fetch
    LOG_ACCESS = False

    def retrieve(self):
        for posting in self.postings:
            if Term.LOG_ACCESS: print(f'{self.term}: {posting}')
            yield posting

# ------------------------------

class Not(BooleanExpression):
    """
        Marker class for NOT operator on sub-expression. The retrieve method raises an exception.
        When used during AND operation, the retrieve method of the sub-expression is called.
    """
    def __init__(self, expression):
        self.expression = expression
    
    def __iter__(self):
        return self.retrieve()

    def retrieve(self):
        raise Exception("NOT operator not allowed at top-level of query")

### Evaluation of: cat AND dog
To evaluate a query like "cat AND dog", we fetch the first entry for each term, that is `1` for cat and `3` for dog. If they are the same, we know that the corresponding document fulfills the predicate. If not, then we read the next entry for the term currently having the smallest doc ID. In our example, we read the next cat posting which is `4`. Again, we have no match, so we progress now postings of dog as it currently has the smallest value. The next posting for dog is `4` which matches with the one of cat; hence, we found our first document and return it (in Python we use generators with `yield` as we also do not want to return all results at once but in batches as the user browses through pages). If we need more results to return, we now fetch the next posting for both terms and repeat. Finally, we find `10` and return it as a second answer. If we need more results, we again fetch the next posting for both terms. But since cat does not have more postings, we can terminate the evaluation and stop iteration (dog still has `12` but we already know from the empty cat postings that it cannot match the query). The following visualizes the approach:

|step|cat (next) |dog (next) | action|
|:-- |:-- |:-- |:-- |
| 1 | `1` | `3` | no match, progress cat |
| 2 | `4` | `3` | no match, progress dog |
| 3 | `4` | `4` | match, return `4` as result, wait to provide next result, and progress both cat and dog |
| 4 | `8` | `10` | no match, progress cat |
| 5 | `10` | `10` | match, return `10` as result, wait to provide next result, and progress both cat and dog |
| 6 | - | `12` | stop iteration as all cat postings are visited; remaining postings in dog cannot fulfill predicate |

### Eavluation of: cat AND NOT(dog)
The "cat AND NOT(dog)" evaluation progress is the same as with the AND flow but results are different (match if cat != dog):

|step|cat (next) |dog (next) | action|
|:-- |:-- |:-- |:-- |
| 1 | `1` | `3` | match, return `1` as result, wait to provide next result, and progress cat |
| 2 | `4` | `3` | match but cat is not smallest, so we progress dog |
| 3 | `4` | `4` | no match as both have the same value, so we progress both cat and dog |
| 4 | `8` | `10` | no match, return `8` as result, wait to provide next result, and progress cat  |
| 5 | `10` | `10` | no match as both have the same value, so we progress both cat and dog |
| 6 | - | `12` | stop iteration as all cat postings are visited; remaining postings in dog cannot fulfill predicate |

Finally, we can apply the same approach for arbitrary nesting of AND and OR operators since each evaluation scheme above yields document IDs again in a sorted manner. As with atomic term queries, we can only support NOT operators if they are part of an AND expression which at least has one sub-expression without a NOT at the top-level (a nested NOT deeper in the sub-expression is not a problem).

In [131]:
class And(BooleanExpression):
    """
        AND-expression with multiple sub-expressions. This operator can handle NOT(expr)-type 
        subexpressions and implements the correct '-' semantics of "cat AND NOT dog".
    """
    def __init__(self, *expressions):
        self.expressions = expressions

        # select expressison that are not Term or that have x._not = False
        self.pos = [e for e in expressions if not isinstance(e, Not)]
        self.neg = [e for e in expressions if isinstance(e, Not)]

    def __iter__(self):
        return self.retrieve()

    def retrieve(self):
        # streams for sub expressions without NOT
        # pos_iters contains the iterators for each sub expression
        # pos_nexts contains the next posting for each sub expression
        pos_iters = [iter(e) for e in self.pos]
        pos_nexts = [next(e) for e in pos_iters]

        # stream for sub expressions with NOT
        # neg_iters contains the iterators for each sub expression
        # neg_nexts contains the next posting for each sub expression
        neg_iters = [iter(e.expression) for e in self.neg]
        neg_nexts = [next(e, None) for e in neg_iters]

        # iterate until one pos_nexts element is None
        while None not in pos_nexts:
            # get smallest value from pos_nexts and neg_nexts, ignoring None values in neg_nexts
            smallest = min(pos_nexts + neg_nexts, key=lambda x: x if x is not None else float('inf'))

            # check if all entries of pos_nexts equal smallest, and no entry in neg_nexts equals smallest
            if all(e is smallest for e in pos_nexts) and smallest not in neg_nexts:
                yield smallest
            
            # for each entry in pos_nexts and neg_nexts, fetch next item if entry equals smallest
            for i, e in enumerate(pos_nexts):
                if e is smallest:
                    pos_nexts[i] = next(pos_iters[i], None)
            for i, e in enumerate(neg_nexts):
                if e is smallest:
                    neg_nexts[i] = next(neg_iters[i], None)

### Evaluation of: cat OR dog
The OR-operator is implemented similarly, but the iteration returns every time the smallest entry from a sub-expression. For the example above, the OR-operator would first return `1`, progress cat, return `3`, progress dog, return `4`, progress both cat and dog, return `8`, progress cat, return `10`, progress both cat and dog, and finally return `12`. The evaluation stops when all postings are consumed.

In [132]:
class Or(BooleanExpression):
    """
        OR-expression with multiple sub-expressions. This operator cannot handle NOT(expr)-type subexpressions
    """
    def __init__(self, *expressions):
        # check that there are no NOT(expr)-type subexpressions
        if any(isinstance(e, Not) for e in expressions):
            raise Exception("OR-expression cannot handle NOT(expr)-type subexpressions")
        self.expressions = expressions
    
    def __iter__(self):
        return self.retrieve()

    def retrieve(self):
        iters = [iter(e) for e in self.expressions]
        nexts = [next(e, None) for e in iters]

        while not all(e is None for e in nexts):
            # get smallest value from nexts, ignoring None values
            smallest = min(nexts, key=lambda x: x if x is not None else float('inf'))
            yield smallest
            
            # for each entry in nexts, fetch next item if entry equals smallest
            for i, e in enumerate(nexts):
                if e is smallest:
                    nexts[i] = next(iters[i], None)

## Putting all together (including a query parser)

In [134]:
retriever = BooleanRetriever(collection.load())

queries = {
    'cat AND dog': ['cat', 'dog', 'cat AND dog'],
    'cat OR dog': ['cat', 'dog', 'cat OR dog'],
    'horse OR bird': ['horse', 'bird', 'horse OR bird'],
    'cat AND NOT(dog)': ['cat', 'dog', 'cat AND NOT(dog)'],
    'horse AND cat AND NOT(bird)': ['horse', 'cat', 'bird', 'horse AND cat', 'horse AND cat AND NOT(bird)'],
    '(cat AND dog) OR (horse AND cat AND NOT(bird))': ['cat', 'dog', 'horse', 'bird', 'cat AND dog', 'horse AND cat', 'horse AND cat AND NOT(bird)', '(cat AND dog) OR (horse AND cat AND NOT(bird))'],
    '(cat OR dog) AND (horse OR bird)': ['cat', 'dog', 'cat OR dog', 'horse OR bird', '(cat OR dog) AND (horse OR bird)'],
    '(cat OR dog) AND NOT(horse OR bird)': ['cat', 'dog', 'cat OR dog', 'horse OR bird', '(cat OR dog) AND NOT(horse OR bird)']    
}

def search_boolean_stream(query: str):
    print_table([[q, list(retriever.search(q))] for q in queries[query]], ['query', 'result'], format="text")

# interactive selection of scenario
widgets.interact(search_boolean_stream, 
    query=widgets.Dropdown(options=list(queries.keys())), 
);

interactive(children=(Dropdown(description='query', options=('cat AND dog', 'cat OR dog', 'horse OR bird', 'ca…

## We can evaluate queries efficiently with generators

Generators are great to prevent evaluation of results that are not needed. Assume the user is querying with "(cat OR dog) AND NOT(horse OR bird)" which generates a lot of results. Rather than returning hundreds of results at once, a user may want to browse through results page-by-page. Our generators do exactly this; even more, we only read postings that we need to produce the results for each batch returned to the users as they browse through results.

In [135]:
query = "cat AND dog"
# query = "(cat OR dog) AND NOT(horse OR bird)"

result = retriever.search(query, logging = True)
print(f"retrieving first 2 documents for {query}")
print(list(islice(result, 2)))

print("\nretrieving next 2 documents")
print(list(islice(result, 2)))

retrieving first 2 documents for cat AND dog
cat: 1
dog: 2
cat: 5
dog: 5
cat: 6
dog: 7
cat: 16
dog: 9
dog: 12
dog: 14
dog: 17
cat: 17
[5, 17]

retrieving next 2 documents
cat: 21
dog: 18
dog: 20
dog: 21
cat: 23
dog: 28
cat: 26
cat: 28
[21, 28]
